In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
class Linear(keras.layers.Layer):
    """y = w.x + b"""

    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim, units), dtype="float32"),
            trainable=True,
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"), trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [3]:
linear_layer = Linear(units=4, input_dim=2)
linear_layer.weights

[<tf.Variable 'Variable:0' shape=(2, 4) dtype=float32, numpy=
 array([[ 0.04436712,  0.02361266, -0.08837948, -0.02131345],
        [ 0.005831  ,  0.02655473,  0.01400602, -0.05616408]],
       dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]

In [4]:
import inspect
from tensorflow.keras.models import Model
inspect.getmro(Model)

(keras.engine.training.Model,
 keras.engine.base_layer.Layer,
 tensorflow.python.module.module.Module,
 tensorflow.python.training.tracking.autotrackable.AutoTrackable,
 tensorflow.python.training.tracking.base.Trackable,
 keras.utils.version_utils.LayerVersionSelector,
 keras.utils.version_utils.ModelVersionSelector,
 object)

In [5]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers

class MyModel(tf.keras.Model):

  def __init__(self, num_classes=10):
    super(MyModel, self).__init__(name='my_model')
    self.num_classes = num_classes
    self.dense_1 = layers.Dense(32, activation='relu')
    self.dense_2 = layers.Dense(num_classes, activation='sigmoid')

  def call(self, inputs): 
    x = self.dense_1(inputs)
    return self.dense_2(x)

model = MyModel(num_classes=10)

model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

@tf.function
def trace():
  data = np.random.random((1, 32))
  model(data)


logdir = "trace_log"
writer = tf.summary.create_file_writer(logdir)
tf.summary.trace_on(graph=True, profiler=True)
# Forward pass
trace()
with writer.as_default():
  tf.summary.trace_export(name="model_trace", step=0, profiler_outdir=logdir)

Instructions for updating:
use `tf.profiler.experimental.start` instead.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.


## Koopman

In [6]:
class DenseResBlock(keras.layers.Layer):
    def __init__(self,
                 n_inputs=128,
                 num_hidden=4,
                 hidden_config=dict(activation='relu',
                                    kernel_initializer='he_normal'),
                 output_config=dict(activation=None,
                                    kernel_initializer='he_normal'),
                 **kwargs):
        super().__init__(**kwargs)
        self.n_inputs = n_inputs
        self.num_hidden = num_hidden
        self.layers = [keras.layers.Dense(n_inputs,
                                          name='hidden{}'.format(i),
                                          **hidden_config)
                       for i in range(num_hidden)]
        self.layers.append(keras.layers.Dense(n_inputs,
                                              name='output',
                                              **output_config))
    def call(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x) # recurrently apply the hidden layers
        return inputs + x  # Add input to output

In [8]:
from NetworkArch import NetworkArch

In [11]:
model = NetworkArch()
model.build([128,21,51])
model.summary()

ValueError: Exception encountered when calling layer "dense_res_block" (type DenseResBlock).

in user code:

    File "c:\Users\websi\Downloads\PINN experiment\PDEKoopman2-master\architecture\DenseResBlock.py", line 51, in call  *
        return inputs + x  # Add input to output

    ValueError: Dimensions must be equal, but are 51 and 128 for '{{node dense_res_block/add}} = AddV2[T=DT_FLOAT](Reshape, dense_res_block/output/BiasAdd)' with input shapes: [2688,51], [2688,128].


Call arguments received by layer "dense_res_block" (type DenseResBlock):
  • inputs=tf.Tensor(shape=(2688, 51), dtype=float32)